### TRMM PNG monitoring

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import xarray as xray
from netCDF4 import MFDataset
from datetime import datetime, timedelta
from glob import glob
import palettable
import folium
from folium.map import FeatureGroup

In [3]:
domain = {'latmin':-13, 'lonmin':155., 'latmax':-5, 'lonmax':168.}

In [4]:
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

In [5]:
def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xray.open_dataset(path) as ds:
            # transform_func shdset_clim = dset_clim.sel(lat=slice(domain['latmin'], domain['latmax']), could do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds)
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
            ds.load()
            return ds

    #paths = sorted(glob(files))
    datasets = [process_one_path(p) for p in files]
    combined = xray.concat(datasets, dim)
    return combined

In [6]:
dpath = '/Users/nicolasf/data/TRMM/daily/'
climpath = '/Users/nicolasf/data/TRMM/climatology/daily/2001_2015/'

In [7]:
today = datetime.utcnow()

In [8]:
lag = 3

In [9]:
trmm_date = today - timedelta(days=lag)

In [10]:
ndays = 90

In [11]:
realtime = pd.date_range(start=trmm_date - timedelta(days=ndays-1), end=trmm_date)

In [12]:
lfiles = []
for d in realtime: 
    fname  = dpath + "3B42RT_daily.{}.nc".format(d.strftime("%Y.%m.%d"))
    lfiles.append(fname)

In [13]:
dset_realtime = read_netcdfs(lfiles, 'time')

In [14]:
dset_realtime = dset_realtime.sel(lat=slice(domain['latmin'], domain['latmax']), \
                                  lon=slice(domain['lonmin'], domain['lonmax']))

In [15]:
clim_files = []
for d in realtime: 
    fname  = climpath + "3B42_daily.{}.nc".format(d.strftime("%m.%d"))
    clim_files.append(fname)

In [16]:
dset_clim = read_netcdfs(clim_files, 'time')

In [17]:
dset_clim = dset_clim.sel(lat=slice(domain['latmin'], domain['latmax']), \
                          lon=slice(domain['lonmin'], domain['lonmax']))

In [18]:
dset_clim

<xarray.Dataset>
Dimensions:  (lat: 32, lon: 52, time: 90)
Coordinates:
  * lon      (lon) float32 155.125 155.375 155.625 155.875 156.125 156.375 ...
  * lat      (lat) float32 -12.875 -12.625 -12.375 -12.125 -11.875 -11.625 ...
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
Data variables:
    trmm     (time, lat, lon) float64 6.891 7.087 8.437 8.698 8.149 8.316 ...

### reads in the location for the virtual stations

In [19]:
Virtual_Stations = pd.read_csv('../data/station_list_Solomon_lat_lon.csv', index_col=0)

In [20]:
Virtual_Stations.tail()

,name_primary,name_secondary,latitude,longitude
33,Gizo,NaN,-8.1000,156.8500
36,Barora,NaN,-8.0333,157.5833
37,Seghe,NaN,-8.5667,157.8833
40,Gibiri,AutoRain,-7.7058,156.7256
41,Mbeulah,AutoRain,-8.2632,157.4279


In [21]:
dsums = {}

for i, row in Virtual_Stations.iterrows(): 
    stn_name = row['name_primary']
    lat_V = row['latitude']
    lon_V = row['longitude']

    
    clim_ts = dset_clim.sel(lat=lat_V, lon=lon_V, method='nearest')['trmm']
    realtime_ts = dset_realtime.sel(lat=lat_V, lon=lon_V, method='nearest')['trmm']
    df_ts = realtime_ts.to_dataframe()[['trmm']]
    df_ts.loc[:,'clim'] = clim_ts.data
    df_ts.columns = ['observed','climatology']
    sums = df_ts.sum()
    dsums[stn_name] = sums

In [22]:
df_dsums = pd.DataFrame(dsums)

In [23]:
df_dsums = df_dsums.T

In [24]:
df_dsums = df_dsums.astype(np.int)

In [25]:
df_dsums.head()

,observed,climatology
Afio,609,931
Atoifi,987,1148
Atori,688,967
Avu Avu,590,849
Barora,582,883


In [26]:
cities = {}
for i, row in Virtual_Stations.iterrows():         
    stn_name = row['name_primary']
    lat_V = row['latitude']
    lon_V = row['longitude']
    cities[stn_name] = (lat_V, lon_V)

## map with Folium

In [27]:
Virtual_Stations.tail()

,name_primary,name_secondary,latitude,longitude
33,Gizo,NaN,-8.1000,156.8500
36,Barora,NaN,-8.0333,157.5833
37,Seghe,NaN,-8.5667,157.8833
40,Gibiri,AutoRain,-7.7058,156.7256
41,Mbeulah,AutoRain,-8.2632,157.4279


In [28]:
fmap = folium.Map(location=[-9, 163], zoom_start=6, min_lon = 0., max_lon = 360., detect_retina=True)

### add the background layers

In [29]:
add = '/tile/{z}/{y}/{x}'

ESRI = {}
ESRI['World Imagery'] = 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer'
ESRI['World Topography'] = 'http://services.arcgisonline.com/arcgis/rest/services/World_Topo_Map/MapServer'
ESRI['Street Map'] = 'http://services.arcgisonline.com/arcgis/rest/services/World_Street_Map/MapServer'

for tile_name in ['World Imagery','World Topography','Street Map']:
    tile_url = ESRI[tile_name]
    tile_url += add
    fmap.add_tile_layer(name=tile_name,
                     tiles=tile_url, attr='ESRI')

### create a feature group which will include the stations markers 

In [30]:
Solomon_stations = FeatureGroup(name='Show Solomon Islands Stations')

In [31]:
size_icon = (188, 325)

### loops over the stations 

In [32]:
for i in Virtual_Stations.index:
    
    data = Virtual_Stations.ix[i]
    
    name_primary = data.name_primary
    
    obs = df_dsums.ix[name_primary].values[0]
    clim = df_dsums.ix[name_primary].values[1]
    
    anom = (obs / clim) * 100
    
    html = """
    <font face="verdana">
    <div align='center'><h4>
    Last 90 days estimates [TRMM/TMPA] for {}</h4>
    observed = <b>{}</b> mm <br>
    climatology = <b>{}</b> mm <br>
    percent. of normal = <b>{:4.2f}<b> %
    <br><br>
    <a href="https://cdn.rawgit.com/nicolasfauchereau/SIMS/master/TRMM/www/Virtual_Station_{}_90ndays.html" target="_blank">click here for the time-series</a>
    </font>
    """.format(name_primary, obs, clim, anom, name_primary)
        
    iframe = folium.element.IFrame(html=html, width=300, height=170)
    
    popup = folium.Popup(iframe, max_width=600)
    
    if anom <= 40: 
        icon = folium.features.CustomIcon('https://raw.githubusercontent.com/nicolasfauchereau/SIMS/master/TRMM/images/icons/orange.png',\
                                         icon_size=(size_icon[0]*0.05, size_icon[1]*0.05))
        marker = folium.map.Marker([data.latitude, data.longitude], icon=icon, popup=popup)   
    elif (anom > 40) & (anom <= 80):
        icon = folium.features.CustomIcon('https://raw.githubusercontent.com/nicolasfauchereau/SIMS/master/TRMM/images/icons/yellow.png',\
                                         icon_size=(size_icon[0]*0.1, size_icon[1]*0.1))
        marker = folium.map.Marker([data.latitude, data.longitude], icon=icon, popup=popup)   
    elif (anom > 80) & (anom < 120):
        icon = folium.features.CustomIcon('https://raw.githubusercontent.com/nicolasfauchereau/SIMS/master/TRMM/images/icons/green.png',\
                                         icon_size=(size_icon[0]*0.15, size_icon[1]*0.15))
        marker = folium.map.Marker([data.latitude, data.longitude], icon=icon, popup=popup)  
    elif (anom >= 120) & (anom < 160):
        icon = folium.features.CustomIcon('https://raw.githubusercontent.com/nicolasfauchereau/SIMS/master/TRMM/images/icons/blue.png',\
                                         icon_size=(size_icon[0]*0.2, size_icon[1]*0.2))
        marker = folium.map.Marker([data.latitude, data.longitude], icon=icon, popup=popup)  
    elif anom > 160:
        icon = folium.features.CustomIcon('https://raw.githubusercontent.com/nicolasfauchereau/SIMS/master/TRMM/images/icons/deepblue.png',\
                                         icon_size=(size_icon[0]*0.25, size_icon[1]*0.25))
        marker = folium.map.Marker([data.latitude, data.longitude], icon=icon, popup=popup)  
    
#     c = folium.CircleMarker(location=[data.latitude, data.longitude], popup=popup, color='red', fill_color='red', \
#                         fill_opacity=0.4, radius=5000)
    
#     c  = folium.Marker(location=[data.latitude, data.longitude], popup=popup)
    
    # add to fature group
    Solomon_stations.add_children(marker)

In [33]:
fmap.add_children(Solomon_stations)
fmap.add_children(folium.map.LayerControl())

In [34]:
fmap.save('./interactive_map_Solomon.html')

In [35]:
!open ./interactive_map_Solomon.html